# Introduction to Machine Learning 

## Exercise 4 - Multi-Class Classification

This notebook covers a Python-based solution for the third programming exercise of the machine learning class on Coursera.  Please refer to the [exercise text](https://github.com/jdwittenauer/ipython-notebooks/blob/master/exercises/ML/ex3.pdf) for detailed descriptions and equations.

For this exercise we'll use logistic regression to recognize hand-written digits (0 to 9).  We'll be extending the implementation of linear regression we wrote in Exercise 2 and apply it to one-vs-all classification.  

Read the sections *Introduction* and 1.1 as well as 1.2 of the detailed exercise description.

Then get started by loading the data set.  The data set is in a MATLAB's native format, so to load it in Python we need to use a SciPy library.

If you want to refresh your knoweledge of Neural Networks in the context of this exercise check out sections 2 of the exercise descriptions.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.io import loadmat
%matplotlib inline

# Load and display data using loadmat

In [2]:
# Determin the dimensionality of your data, to varify that all samples got loaded correctly

Great, we've got our data loaded.  The images are represented in martix X as a 400-dimensional vector (of which there are 5,000 of them).  The 400 "features" are grayscale intensities of each pixel in the original 20 x 20 image.  The class labels are in the vector y as a numeric class representing the digit that's in the image.

The first task is to modify our linear regression implementation into a logistic regression in completely vectorized form (i.e. no "for" loops).  We need vectorized code, because in addition to being short and concise, is able to take advantage of linear algebra optimizations and is typically much faster than iterative code.

If you take a look at our cost function implementation from the Exercise 2 Solution, you can see that it's already vectorized!  So we can re-use that same implementation here and only need to take care of modifying it into a classfication.
We will use the Sigmoid function as our activation function, so start by implementing that.

In [3]:
def sigmoid(z):
    # TODO

SyntaxError: unexpected EOF while parsing (<ipython-input-3-8ae6f53a3378>, line 2)

Now continue with the cost function. You can find the precise mathematical formula to be implemented here in section 1.3.1 of the detailed exercise description.

In [4]:
def cost(theta, X, y, learningRate):
    # TODO

Next we need the function that computes the gradient.  Again, we already defined this in the previous exercise, only in this case we want to get rid of the "for" loop in the update step.  Here's the original code for reference:

In [5]:
def gradient_with_loop(theta, X, y, learningRate):
    theta = np.matrix(theta)
    X = np.matrix(X)
    y = np.matrix(y)
    
    parameters = int(theta.ravel().shape[1])
    grad = np.zeros(parameters)
    
    error = sigmoid(X * theta.T) - y
    
    for i in range(parameters):
        term = np.multiply(error, X[:,i])
        
        if (i == 0):
            grad[i] = np.sum(term) / len(X)
        else:
            grad[i] = (np.sum(term) / len(X)) + ((learningRate / len(X)) * theta[:,i])
    
    return grad

In our new version we're going to pull out the "for" loop and compute the gradient for all the values of theta at once using linear algebra.  To follow the math behind the transformation, refer to the exercise 3 text section 1.3.2

In [6]:
def gradient(theta, X, y, learningRate):
   # TODO

Now that we've defined our cost and gradient functions, it's time to build a classifier.  For this task we've got 10 possible classes (one for every digit), and since logistic regression is only able to distiguish between 2 classes at a time, we need a strategy to deal with the multi-class scenario.

Read section 1.4 of the detailed exercise description to understand the approach we will take.

The idea: We immplement a one-vs-all classification approach, where a label with k different classes results in k classifiers, each one deciding between "class i" and "not class i" (i.e. any class other than i).  We're going to wrap the classifier training up in one function that computes the final weights for each of the 10 classifiers and returns the weights as a k X (n + 1) array, where n is the number of parameters.

In [7]:
from scipy.optimize import minimize

def one_vs_all(X, y, num_labels, learning_rate):
    rows = X.shape[0]
    params = X.shape[1]
    
    # k X (n + 1) array for the parameters of each of the k classifiers
    # TODO
    
    # insert a column of ones at the beginning for the intercept term (same as in Exercise 2)
    # TODO
    
    # iterate over all labels (labels are 1-indexed instead of 0-indexed)
    for i in range(1, num_labels + 1):
        #TODO 
        
        # minimize the objective function
        # don't do this by hand! Use scipys optimization API
        #TODO 
    
    return all_theta

A few things to note here...first, we'are adding an extra parameter to theta (along with a column of ones to the training data) to account for the intercept term.  Second, we're transforming y from a class label to a binary value for each classifier (either is class i or is not class i).  Finally, we're using SciPy's newer optimization API to minimize the cost function for each classifier.  The API takes an objective function, an initial set of parameters, an optimization method, and a jacobian (gradient) function if specified.  The parameters found by the optimization routine are then assigned to the parameter array.

One of the more challenging parts of implementing vectorized code is getting all of the matrix interactions written correctly, so I find it useful to do some sanity checks by looking at the shapes of the arrays/matrices I'm working with and convincing myself that they're sensible :)

In [8]:
# Do some checking on our data

rows = data['X'].shape[0]
params = data['X'].shape[1]

all_theta = np.zeros((10, params + 1))

X = np.insert(data['X'], 0, values=np.ones(rows), axis=1)

theta = np.zeros(params + 1)

y_0 = np.array([1 if label == 0 else 0 for label in data['y']])
y_0 = np.reshape(y_0, (rows, 1))

X.shape, y_0.shape, theta.shape, all_theta.shape

((5000L, 401L), (5000L, 1L), (401L,), (10L, 401L))

These all appear to make sense.  Note that theta is a one-dimensional array, so when it gets converted to a matrix in the code that computes the gradient, it turns into a (1 X 401) matrix.  Let's also check the class labels in y to make sure they look like what we're expecting.

In [9]:
np.unique(data['y'])

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10], dtype=uint8)

Let's make sure that our training function actually runs, and we get some sensible outputs, before going any further.

In [10]:
all_theta = one_vs_all(data['X'], data['y'], 10, 1)
all_theta

array([[ -5.79312170e+00,   0.00000000e+00,   0.00000000e+00, ...,
          1.22140973e-02,   2.88611969e-07,   0.00000000e+00],
       [ -4.91685285e+00,   0.00000000e+00,   0.00000000e+00, ...,
          2.40449128e-01,  -1.08488270e-02,   0.00000000e+00],
       [ -8.56840371e+00,   0.00000000e+00,   0.00000000e+00, ...,
         -2.59241796e-04,  -1.12756844e-06,   0.00000000e+00],
       ..., 
       [ -1.32641613e+01,   0.00000000e+00,   0.00000000e+00, ...,
         -5.63659404e+00,   6.50939114e-01,   0.00000000e+00],
       [ -8.55392716e+00,   0.00000000e+00,   0.00000000e+00, ...,
         -2.01206880e-01,   9.61930149e-03,   0.00000000e+00],
       [ -1.29807876e+01,   0.00000000e+00,   0.00000000e+00, ...,
          2.60651472e-04,   4.22693052e-05,   0.00000000e+00]])

We're now ready for the final step - using the trained classifiers to predict a label for each image.  For this step we're going to compute the class probability for each class, for each training instance (using vectorized code of course!) and assign the output class label as the class with the highest probability.
Check out section 1.4.1 of the exercise description for more details.

In [11]:
def predict_all(X, all_theta):
    #TODO 
    return h_argmax

Now we can use the predict_all function to generate class predictions for each instance and see how well our classifier works.

In [12]:
y_pred = predict_all(data['X'], all_theta)
correct = [1 if a == b else 0 for (a, b) in zip(y_pred, data['y'])]
accuracy = (sum(map(int, correct)) / float(len(correct)))
print 'accuracy = {0}%'.format(accuracy * 100)

accuracy = 97.58%


Almost 98% isn't too bad!  That's all for exercise 3.  In the next exercise, we'll look at how to implement a feed-forward neural network from scratch.